In [139]:
import functions_ml as fml
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import hamming_loss, f1_score, jaccard_score, accuracy_score, multilabel_confusion_matrix, classification_report

Depois posso separar uma análise para problemas que não haviam no teste (30 plantas, 5 produtos, etc) (o dataset mantém o índice no y_test, basta fazer merge das colunas para filtrar plantas, produtos e dps dar drop)

# Oracle

In [140]:
# Load model
TOLERANCE = 0.1
with open(f'trained_models/oracle_{TOLERANCE}percent.pkl', 'rb') as f:
    model = pickle.load(f)

# Get train set features

In [141]:
results = pd.read_csv('datasets/instances_results.csv')
features = pd.read_csv('datasets/multi_plant_instance_features.csv')
dataset = fml.create_dataset(features, results)
dataset = fml.create_multi_label_target(dataset, TOLERANCE / 100)
target_col = [t for t in dataset.columns if t.startswith('RF')]
X_train = dataset.drop(columns=target_col + ['instance'])
X_train = fml.binary_feature_selection(X_train)
# X_train = fml.multi_class_feature_selection(X_train)
del results, features, dataset, target_col

# Test set ("new instances")

In [142]:
TEST_TOLERANCE_LIMIT = TOLERANCE / 100
test_results = pd.read_csv('datasets/test_instances_results.csv')
test_features = pd.read_csv('datasets/test_instances_features.csv')
test_set = fml.create_dataset(test_features, test_results)
test_set = fml.create_multi_label_target(test_set, TEST_TOLERANCE_LIMIT)

In [143]:
target_cols = [t for t in test_set.columns if t.startswith('RF')]
for t in target_cols:
    print(t, ':', test_set[t].sum())

RF_1_0 : 60
RF_2_0 : 60
RF_2_1 : 62
RF_3_0 : 57
RF_3_1 : 59
RF_3_2 : 53
RF_4_0 : 60
RF_4_1 : 63
RF_4_2 : 58
RF_4_3 : 58
RF_6_0 : 67
RF_6_1 : 60
RF_6_2 : 60
RF_6_3 : 68
RF_6_4 : 67
RF_6_5 : 56
RF_T_0 : 82


# Conjuntos de instâncias
Separar em tamanhos vistos e não vistos, pois estes podem ter comportamentos diferentes dos planos dos modelo

In [144]:
seen_nplants = [2, 4, 6, 15, 20]
seen_nproducts = [10, 60, 120]
unseen_nproducts = [p for p in test_set['num_products'].unique() if p not in seen_nproducts and p > max(seen_nproducts) or p < min(seen_nproducts)] # [np.int64(30), np.int64(150), np.int64(5)]
unseen_nplants = [p for p in test_set['num_plants'].unique() if p not in seen_nplants and p > max(seen_nplants)]  # [np.int64(30), np.int64(10)]

In [145]:
test_set = test_set.loc[~(test_set['num_products'].isin(unseen_nproducts)) &
                        ~(test_set['num_plants'].isin(unseen_nplants))]

In [146]:
test_set['instance'].str[:5].value_counts()

instance
VAL00    16
VAL01    16
VAL11    16
VAL10    16
Name: count, dtype: int64

In [147]:
test_set['num_plants'].unique()

array([10,  2, 20,  6])

In [148]:
test_set[['instance', 'num_plants', 'num_products']]

,instance,num_plants,num_products
83,VAL00_12_10_30,10,30
15,VAL01_12_2_30,2,30
58,VAL11_12_2_120,2,120
64,VAL01_12_2_60,2,60
94,VAL01_12_20_120,20,120
...,...,...,...
11,VAL00_12_2_60,2,60
57,VAL11_12_10_60,10,60
1,VAL00_12_20_120,20,120
107,VAL00_12_2_10,2,10


# Preprocessing (deve ser igual ao aplicado no desenvolvimento)

In [149]:
X_test = test_set[X_train.columns]
y_test = test_set[target_cols]

## Predict

In [150]:
y_pred = pd.DataFrame(model.predict(X_test), columns=y_test.columns, index=y_test.index)
probabilities = pd.DataFrame(model.predict_proba(X_test), columns=y_test.columns, index=y_test.index)

In [151]:
hamming = hamming_loss(y_test, y_pred)
print("Hamming Loss:", hamming)

f1 = f1_score(y_test, y_pred, average='micro')
print("Micro-Averaged F1 Score:", f1)

jaccard = jaccard_score(y_test, y_pred, average='samples')
print("Jaccard Similarity Score:", jaccard)

subset_accuracy = accuracy_score(y_test, y_pred)
print("Subset Accuracy:", subset_accuracy)

Hamming Loss: 0.09834558823529412
Micro-Averaged F1 Score: 0.9042076991942704
Jaccard Similarity Score: 0.561359723813768
Subset Accuracy: 0.34375


In [152]:
print(classification_report(y_test, y_pred, target_names=y_test.columns, zero_division="warn"))

              precision    recall  f1-score   support

      RF_1_0       0.96      0.70      0.81        33
      RF_2_0       1.00      0.76      0.86        33
      RF_2_1       1.00      0.82      0.90        38
      RF_3_0       0.87      0.87      0.87        31
      RF_3_1       1.00      0.88      0.94        34
      RF_3_2       1.00      0.84      0.91        31
      RF_4_0       0.97      0.88      0.92        33
      RF_4_1       1.00      0.89      0.94        35
      RF_4_2       1.00      0.94      0.97        33
      RF_4_3       1.00      0.79      0.88        33
      RF_6_0       1.00      0.91      0.95        34
      RF_6_1       1.00      0.91      0.95        33
      RF_6_2       0.86      0.83      0.85        36
      RF_6_3       0.86      0.84      0.85        38
      RF_6_4       0.97      0.82      0.89        40
      RF_6_5       0.94      0.89      0.91        35
      RF_T_0       0.97      0.93      0.95        42

   micro avg       0.96   

/home/lucasmalheiros/Documents/Pesquisa Operacional/tcc-relax-and-fix/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [153]:
method_evaluation_results = []
for method in y_test.columns:
    # Calculate true positives (correct assignments)
    true_positives = ((y_pred == 1) & (y_test == 1))[method].sum()

    # Calculate false positives (predicted as 1 but actual is 0)
    false_positives = ((y_pred == 1) & (y_test == 0))[method].sum()

    # Calculate false negatives (predicted as 0 but actual is 1)
    false_negatives = ((y_pred == 0) & (y_test == 1))[method].sum()

    # Calculate Precision
    if (true_positives + false_positives) > 0:
        precision = true_positives / (true_positives + false_positives)
    else:
        precision = 0

    # Calculate Recall
    if (true_positives + false_negatives) > 0:
        recall = true_positives / (true_positives + false_negatives)
    else:
        recall = 0 

    # Calculate F1-Score
    if (precision + recall) > 0:
        f1_score = (2 * precision * recall) / (precision + recall)
    else:
        f1_score = 0

    # Append results to the list
    method_evaluation_results.append({
        'Method': method,
        'True Positives': true_positives,
        'False Positives': false_positives,
        'False Negatives': false_negatives,
        'Precision (%)': f'{precision * 100 :.2f}',
        'Recall (%)': f'{recall * 100 :.2f}',
        'F1-Score': f'{f1_score * 100 :.2f}'
    })
method_evaluation_results_df = pd.DataFrame(method_evaluation_results)

In [154]:
method_evaluation_results_df

,Method,True Positives,False Positives,False Negatives,Precision (%),Recall (%),F1-Score
0,RF_1_0,23,1,10,95.83,69.70,80.70
1,RF_2_0,25,0,8,100.00,75.76,86.21
2,RF_2_1,31,0,7,100.00,81.58,89.86
3,RF_3_0,27,4,4,87.10,87.10,87.10
4,RF_3_1,30,0,4,100.00,88.24,93.75
5,RF_3_2,26,0,5,100.00,83.87,91.23
6,RF_4_0,29,1,4,96.67,87.88,92.06
7,RF_4_1,31,0,4,100.00,88.57,93.94
8,RF_4_2,31,0,2,100.00,93.94,96.88
9,RF_4_3,26,0,7,100.00,78.79,88.14


In [155]:
print(method_evaluation_results_df.to_latex())

\begin{tabular}{llrrrlll}
\toprule
 & Method & True Positives & False Positives & False Negatives & Precision (%) & Recall (%) & F1-Score \\
\midrule
0 & RF_1_0 & 23 & 1 & 10 & 95.83 & 69.70 & 80.70 \\
1 & RF_2_0 & 25 & 0 & 8 & 100.00 & 75.76 & 86.21 \\
2 & RF_2_1 & 31 & 0 & 7 & 100.00 & 81.58 & 89.86 \\
3 & RF_3_0 & 27 & 4 & 4 & 87.10 & 87.10 & 87.10 \\
4 & RF_3_1 & 30 & 0 & 4 & 100.00 & 88.24 & 93.75 \\
5 & RF_3_2 & 26 & 0 & 5 & 100.00 & 83.87 & 91.23 \\
6 & RF_4_0 & 29 & 1 & 4 & 96.67 & 87.88 & 92.06 \\
7 & RF_4_1 & 31 & 0 & 4 & 100.00 & 88.57 & 93.94 \\
8 & RF_4_2 & 31 & 0 & 2 & 100.00 & 93.94 & 96.88 \\
9 & RF_4_3 & 26 & 0 & 7 & 100.00 & 78.79 & 88.14 \\
10 & RF_6_0 & 31 & 0 & 3 & 100.00 & 91.18 & 95.38 \\
11 & RF_6_1 & 30 & 0 & 3 & 100.00 & 90.91 & 95.24 \\
12 & RF_6_2 & 30 & 5 & 6 & 85.71 & 83.33 & 84.51 \\
13 & RF_6_3 & 32 & 5 & 6 & 86.49 & 84.21 & 85.33 \\
14 & RF_6_4 & 33 & 1 & 7 & 97.06 & 82.50 & 89.19 \\
15 & RF_6_5 & 31 & 2 & 4 & 93.94 & 88.57 & 91.18 \\
16 & RF_T_0 & 39 &

## Model methods choice

### Select top K

In [156]:
def select_top_k_methods(probabilities_df, y_test, top_k, keep_probabilities=False):
    """
    Select the top-K methods based on probabilities.

    Parameters:
        probabilities_df (DataFrame): DataFrame of probabilities for each method.
        y_test (DataFrame): DataFrame with true labels for each method.
        top_k (int): Number of top methods to select.
        keep_probabilities (bool): If True, keep the probabilities of the top-K methods instead of marking them as 1.
    
    Returns:
        DataFrame: A DataFrame with either binary values or probabilities for the top-K methods.
    """
    # Ensure the input is a pandas DataFrame
    probabilities = probabilities_df.values  # Convert to numpy array for processing
    
    # Rank the methods based on probabilities in descending order
    ranked_methods = np.argsort(-probabilities, axis=1)
    
    # Initialize an array for top-K selection
    top_k_methods = np.zeros_like(probabilities)
    
    # Process each instance
    for i in range(probabilities.shape[0]):  # Loop over each row
        top_indices = ranked_methods[i, :top_k]  # Get top-K indices for this instance
        if keep_probabilities:
            top_k_methods[i, top_indices] = probabilities[i, top_indices]  # Assign probabilities
        else:
            top_k_methods[i, top_indices] = 1  # Mark as selected

    # Convert back to a DataFrame for consistency
    top_k_methods_df = pd.DataFrame(top_k_methods, columns=probabilities_df.columns, index=probabilities_df.index)
    return top_k_methods_df

In [157]:
def evaluate_top_k_predictions(top_k_methods_df, y_test, top_k):
    """Evaluate top-K predictions against the true test set."""
    # Element-wise multiplication to find correct predictions
    correct_predictions = top_k_methods_df * y_test
    # Sum the total number of correct predictions
    total_correct = correct_predictions.sum().sum()  # Sum across all rows and columns
    # Accuracy as a percentage
    total_possible = top_k * len(top_k_methods_df)  # Total number of actual positive labels
    accuracy = (total_correct / total_possible) * 100
    # Precision (if top_k == 1)
    precision = None
    if top_k == 1:
        precision = total_correct / len(y_test) * 100
    # Correct predictions per instance
    correct_per_instance = correct_predictions.sum(axis=1)
    # Identify rows with zero correct predictions
    missed_all = (correct_per_instance == 0).sum()  # Count rows with no correct predictions
    total_instances = len(correct_predictions)  # Total number of rows
    # Percentage of rows with at least one correct prediction
    general_accuracy = (1 - (missed_all / total_instances)) * 100
    # Return metrics
    results = {
        # 'Total Correct Predictions': total_correct,
        # 'Accuracy (%)': accuracy,
        # 'Precision (%)': precision,
        'Missed Rows': missed_all,
        'General Accuracy (%)': general_accuracy
    }
    return results

In [167]:
TOP_K = 1
top_k_methods_df_prob = select_top_k_methods(probabilities, y_test, top_k=TOP_K, keep_probabilities=True)
top_k_methods_df_bin = select_top_k_methods(probabilities, y_test, top_k=TOP_K, keep_probabilities=False)
evaluate_top_k_predictions(top_k_methods_df_bin, y_test, TOP_K)

{'Missed Rows': np.int64(15), 'General Accuracy (%)': np.float64(76.5625)}

In [168]:
# Step 1: Element-wise multiplication to find correct predictions
correct_predictions = top_k_methods_df_bin * y_test

# Step 2: Calculate the number of correct predictions per row
correct_per_row = correct_predictions.sum(axis=1)

# Step 3: Define the threshold (N) and filter rows where correct predictions >= N
N = 0 # Replace with your desired value for N
rows_with_at_least_N_correct = correct_per_row[correct_per_row == N]

# Step 4: Get the percentage of rows meeting the criterion
total_rows = len(correct_per_row)
percentage = (len(rows_with_at_least_N_correct) / total_rows) * 100

# Step 5: Display the results
print(f"Percentage of rows with {N} correct predictions: {percentage:.2f}%")

# Optionally, display these rows for further analysis
rows_with_N_correct_indices = rows_with_at_least_N_correct.index
matching_rows = top_k_methods_df_bin.loc[rows_with_N_correct_indices]
print("\nMatching Rows:")
matching_rows

Percentage of rows with 0 correct predictions: 23.44%

Matching Rows:


,RF_1_0,RF_2_0,RF_2_1,RF_3_0,RF_3_1,RF_3_2,RF_4_0,RF_4_1,RF_4_2,RF_4_3,RF_6_0,RF_6_1,RF_6_2,RF_6_3,RF_6_4,RF_6_5,RF_T_0
110,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
63,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
89,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [169]:
pd.concat([probabilities.loc[rows_with_N_correct_indices], test_set[['instance']].loc[rows_with_N_correct_indices]], axis=1)

,RF_1_0,RF_2_0,RF_2_1,RF_3_0,RF_3_1,RF_3_2,RF_4_0,RF_4_1,RF_4_2,RF_4_3,RF_6_0,RF_6_1,RF_6_2,RF_6_3,RF_6_4,RF_6_5,RF_T_0,instance
110,0.038062,0.261119,0.191141,0.043908,0.142448,0.045185,0.045603,0.056676,0.070397,0.034506,0.055708,0.050182,0.068536,0.053257,0.052619,0.034085,0.042692,VAL11_12_10_30
84,0.495613,0.215145,0.121515,0.050195,0.059166,0.041275,0.041225,0.057110,0.040350,0.034420,0.051965,0.040465,0.055972,0.054174,0.051111,0.035446,0.053384,VAL10_12_6_120
36,0.037508,0.155655,0.141568,0.112428,0.207500,0.150429,0.064357,0.108163,0.444668,0.072603,0.104767,0.091840,0.193179,0.338001,0.248405,0.177253,0.063959,VAL11_12_10_10
75,0.054083,0.235687,0.181762,0.064329,0.232952,0.050516,0.054454,0.108190,0.098887,0.048221,0.112029,0.058506,0.174157,0.217240,0.241078,0.105188,0.074256,VAL10_12_6_30
63,0.065118,0.369561,0.240191,0.050218,0.117155,0.046774,0.049915,0.052990,0.071286,0.031686,0.060880,0.046976,0.064872,0.046318,0.041579,0.028816,0.043114,VAL10_12_10_60
21,0.032124,0.046035,0.061420,0.041702,0.071813,0.048963,0.036194,0.049904,0.048121,0.045666,0.056497,0.060760,0.411597,0.668628,0.863935,0.736515,0.919798,VAL10_12_2_60
89,0.295295,0.369071,0.095076,0.062013,0.088897,0.041438,0.048022,0.119861,0.040548,0.031645,0.047158,0.042058,0.050334,0.042334,0.042408,0.027649,0.047650,VAL10_12_10_120
104,0.602167,0.195664,0.064193,0.060854,0.054034,0.033676,0.044549,0.045817,0.040117,0.033794,0.051585,0.039898,0.048954,0.042367,0.038176,0.029492,0.060154,VAL10_12_20_120
109,0.129784,0.345333,0.225538,0.210989,0.328447,0.049036,0.115734,0.087864,0.108140,0.030836,0.094442,0.057565,0.070627,0.054912,0.044659,0.032260,0.045042,VAL11_12_20_30
24,0.055769,0.201091,0.159050,0.045471,0.157267,0.038964,0.055311,0.090769,0.082081,0.048872,0.084727,0.054301,0.102169,0.117565,0.177128,0.093781,0.071251,VAL11_12_6_30


In [161]:
pd.concat([y_test.loc[rows_with_N_correct_indices], test_set[['instance']].loc[rows_with_N_correct_indices]], axis=1)

,RF_1_0,RF_2_0,RF_2_1,RF_3_0,RF_3_1,RF_3_2,RF_4_0,RF_4_1,RF_4_2,RF_4_3,RF_6_0,RF_6_1,RF_6_2,RF_6_3,RF_6_4,RF_6_5,RF_T_0,instance
110,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,VAL11_12_10_30
84,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,VAL10_12_6_120
36,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,VAL11_12_10_10
75,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,VAL10_12_6_30
63,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,VAL10_12_10_60
21,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,VAL10_12_2_60
89,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,VAL10_12_10_120
104,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,VAL10_12_20_120
109,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,VAL11_12_20_30
24,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,VAL11_12_6_30


In [162]:
top_k_methods_df_prob.loc[rows_with_N_correct_indices]

,RF_1_0,RF_2_0,RF_2_1,RF_3_0,RF_3_1,RF_3_2,RF_4_0,RF_4_1,RF_4_2,RF_4_3,RF_6_0,RF_6_1,RF_6_2,RF_6_3,RF_6_4,RF_6_5,RF_T_0
110,0.000000,0.261119,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
84,0.495613,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
36,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.444668,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
75,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.241078,0.0,0.000000
63,0.000000,0.369561,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
21,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.919798
89,0.000000,0.369071,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
104,0.602167,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
109,0.000000,0.345333,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
24,0.000000,0.201091,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


### Performance what if only 1 method

In [163]:
# Step 1: Initialize a DataFrame to store results
method_evaluation_results = []

# Step 2: Loop through each method
for method in y_test.columns:
    # Create a binary matrix where the current method is always 1 and others are 0
    single_method_matrix = np.zeros_like(y_test, dtype=int)
    single_method_matrix[:, y_test.columns.get_loc(method)] = 1  # Set current method column to 1
    
    # Convert to DataFrame
    single_method_df = pd.DataFrame(single_method_matrix, columns=y_test.columns, index=y_test.index)

    # Calculate true positives (correct assignments)
    true_positives = ((single_method_df == 1) & (y_test == 1))[method].sum()

    # Calculate false positives (predicted as 1 but actual is 0)
    false_positives = ((single_method_df == 1) & (y_test == 0))[method].sum()

    # Calculate false negatives (predicted as 0 but actual is 1)
    false_negatives = ((single_method_df == 0) & (y_test == 1))[method].sum()

    # Calculate Precision
    if (true_positives + false_positives) > 0:
        precision = true_positives / (true_positives + false_positives)
    else:
        precision = 0.0  # Handle division by zero

    # Calculate Recall
    if (true_positives + false_negatives) > 0:
        recall = true_positives / (true_positives + false_negatives)
    else:
        recall = 0.0  # Handle division by zero

    # Append results to the list
    method_evaluation_results.append({
        'Method': method,
        'True Positives': true_positives,
        'False Positives': false_positives,
        'False Negatives': false_negatives,
        'Precision (%)': precision * 100,
        'Recall (%)': recall * 100
    })

In [164]:
# Step 3: Convert results to a DataFrame for easier visualization
method_evaluation_results_df = pd.DataFrame(method_evaluation_results)

# Display the evaluation results
method_evaluation_results_df

,Method,True Positives,False Positives,False Negatives,Precision (%),Recall (%)
0,RF_1_0,33,31,0,51.5625,100.0
1,RF_2_0,33,31,0,51.5625,100.0
2,RF_2_1,38,26,0,59.3750,100.0
3,RF_3_0,31,33,0,48.4375,100.0
4,RF_3_1,34,30,0,53.1250,100.0
5,RF_3_2,31,33,0,48.4375,100.0
6,RF_4_0,33,31,0,51.5625,100.0
7,RF_4_1,35,29,0,54.6875,100.0
8,RF_4_2,33,31,0,51.5625,100.0
9,RF_4_3,33,31,0,51.5625,100.0


In [165]:
print(method_evaluation_results_df.to_latex())

\begin{tabular}{llrrrrr}
\toprule
 & Method & True Positives & False Positives & False Negatives & Precision (%) & Recall (%) \\
\midrule
0 & RF_1_0 & 33 & 31 & 0 & 51.562500 & 100.000000 \\
1 & RF_2_0 & 33 & 31 & 0 & 51.562500 & 100.000000 \\
2 & RF_2_1 & 38 & 26 & 0 & 59.375000 & 100.000000 \\
3 & RF_3_0 & 31 & 33 & 0 & 48.437500 & 100.000000 \\
4 & RF_3_1 & 34 & 30 & 0 & 53.125000 & 100.000000 \\
5 & RF_3_2 & 31 & 33 & 0 & 48.437500 & 100.000000 \\
6 & RF_4_0 & 33 & 31 & 0 & 51.562500 & 100.000000 \\
7 & RF_4_1 & 35 & 29 & 0 & 54.687500 & 100.000000 \\
8 & RF_4_2 & 33 & 31 & 0 & 51.562500 & 100.000000 \\
9 & RF_4_3 & 33 & 31 & 0 & 51.562500 & 100.000000 \\
10 & RF_6_0 & 34 & 30 & 0 & 53.125000 & 100.000000 \\
11 & RF_6_1 & 33 & 31 & 0 & 51.562500 & 100.000000 \\
12 & RF_6_2 & 36 & 28 & 0 & 56.250000 & 100.000000 \\
13 & RF_6_3 & 38 & 26 & 0 & 59.375000 & 100.000000 \\
14 & RF_6_4 & 40 & 24 & 0 & 62.500000 & 100.000000 \\
15 & RF_6_5 & 35 & 29 & 0 & 54.687500 & 100.000000 \\
16 & RF_